#### USA   National Sex Offender Public Website (NSOPW)  
NSOPW provides public access to sex offenders data in 50 states in USA.The code below scrapes website by State name and first letters of first and last names to get offender name, age, profile link and addresses.

https://www.nsopw.gov/search-public-sex-offender-registries


### Import modules

In [37]:
from itertools import product
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import pandas as pd
import re
import time
import traceback

### Global variables

In [38]:
dictOffenders_final = dict()
dictErrors = dict()

### Start driver

In [39]:
driver = webdriver.Chrome()

### Functions

In [40]:

def get_alphabet():
    l1 = list(map(chr,range(97,123)))
    ab = list(product(l1, repeat=2))
    abc = list(product(ab,l1, repeat=1))
    ln = len(abc)
    return abc,ln

def set_field_value(firstNameKey, lastNameKey):
    inputName = driver.find_element(By.XPATH,'//*[@id="firstname"]')
    inputLastName = driver.find_element(By.XPATH,'//*[@id="lastname"]')
    
    inputName.clear()
    inputLastName.clear()
    
    inputName.send_keys(firstNameKey)
    inputLastName.send_keys(lastNameKey)
    
    return f"SET First key {firstNameKey} and LastKey {lastNameKey}"

def set_state_abbr(state_abbr):
    stateList = Select(driver.find_element(By.XPATH,'//*[@id="stateterritory"]'))
    stateList.select_by_value(state_abbr)
    return state_abbr

def show_all_entries():
    showEntries = Select(driver.find_element(By.NAME,'nsopwdt_length'))
    showEntries.select_by_value("-1")
    
    numResults  = driver.find_element(By.XPATH,'//*[@id="nsopwdt_info"]')
    result = re.search('of(.*)entries', numResults.text)
    NumRecords = int(result.group(1))
    return NumRecords



def get_entries_data(num_entries,f,l):
    
    try:
        i = 0
        Ids = []
        Names = []
        Ages = []
        Aliases = []
        Addresses = []
        Links = []
        Characters = []

        for el in range(1,num_entries+1):
            Name = driver.find_element(By.XPATH,'//*[@id="nsopwdt"]/tbody/tr['+str(el)+']/td[1]').text.replace("\n"," ")
            Age = driver.find_element(By.XPATH,'//*[@id="nsopwdt"]/tbody/tr['+str(el)+']/td[2]').text.replace("\n"," ")
            Address = driver.find_element(By.XPATH,'//*[@id="nsopwdt"]/tbody/tr['+str(el)+']/td[4]').text.replace("\n"," ")

            href=driver.find_element(By.XPATH,'//*[@id="nsopwdt"]/tbody/tr['+str(el)+']/td[1]/a').get_attribute('href')
            resultId = re.search('OfndrID=(.*)&AgencyID', href)
            Ids.append(resultId.group(1))
            Names.append(Name)
            Ages.append(Age)
            Addresses.append(Address)
            Links.append(href)
            Characters.append([f,l])
            i=i+1
    except :
        print(traceback.format_exc())

    return Ids,Names,Ages,Addresses,Links,Characters


### Start Selenium

In [41]:

URL = "https://www.nsopw.gov/search-public-sex-offender-registries"
driver.get(URL)

driver.find_element(By.XPATH,'//*[@id="confirmBtn"]').click()
driver.implicitly_wait(10) # seconds

### Run code

In [ ]:
###Main 
alph,length = get_alphabet()
total_cumulative_time =0
total_start_time = time.time()
char_list = list()

for x in range(0,length):
#for x in range(397,length):

    loop_start_time = time.time()
    
    print("".join(alph[x][0]))
    set_field_value("".join(alph[x][0]),alph[x][1])
    
    # Pass state abbreviation(2 letters)
    #driver.implicitly_wait(2)
    stateAbbrv = set_state_abbr('VT')
    
    #time.sleep(1) # seconds
    #driver.implicitly_wait(2)
    driver.find_element(By.XPATH,'//*[@id="searchbynamezip"]').click()
    #driver.implicitly_wait(2) # seconds

    time.sleep(4) # seconds
    #driver.implicitly_wait(4)
    try:
        numEntries = show_all_entries()
    except AttributeError:
        print(f"AttributeError: {alph[x][0]} and {alph[x][1]}")
        char_list.append(["".join(alph[x][0]),alph[x][1]])
        dictErrors[stateAbbrv]=char_list
        pass
    #driver.implicitly_wait(6) # seconds

    
    id,name,age,address,link,character=get_entries_data( numEntries,"".join(alph[x][0]),alph[x][1] )
    #time.sleep(2) # seconds
    #driver.implicitly_wait(2)
    

    
    #Put lists together into dictionary
    nms = list(zip(name,age, address,link,character))
    dictOffenders = dict(zip(id,nms))
    
    #Send data to global dictionary
    dictOffenders_final.update(dictOffenders)
    print(f"total records processed {len(dictOffenders_final)}")
    
    #go back to search page
    driver.find_element(By.XPATH,'//*[@id="nsopwsearchfilter"]').click()
    time.sleep(2) # seconds
    
    loop_end_time = time.time()-loop_start_time
    total_cumulative_time = total_cumulative_time+loop_end_time
    #print(f"loop end time {loop_end_time}")
    print(f"total cumulative time  {total_cumulative_time}")
    print(f"----------------------------------------------------")
    
total_elapsed_time = time.time()-total_start_time
print(f"total elapsed time  {total_elapsed_time}")

df = pd.DataFrame.from_dict(dictOffenders_final,orient='index',columns = ['Name','Age','Address','Link','Characters'])
df.to_csv('Vermont_Offenders_06112024.csv')